<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/combining_model_%26_voting_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# import necessary packages
! pip3 install mapclassify
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [45]:
# Load data
md = pd.read_csv('./model_data.csv') # Load in model_data (county data)
vdf = pd.read_csv('./county_votes_allyears.csv') # Load historical voting data
pop = pd.read_csv('/content/df_melt.csv') # Load the population data

In [46]:
vdf.drop(columns=['Unnamed: 0'], inplace=True)
vdf.head()

,year,state,county_fips,county_name,democrat_votes,republican_votes,total_votes
0,2000,VIRGINIA,51001,ACCOMACK,5092,6352,11444
1,2000,VIRGINIA,51003,ALBEMARLE,16255,18291,34546
2,2000,VIRGINIA,51005,ALLEGHANY,2214,2808,5022
3,2000,VIRGINIA,51007,AMELIA,1754,2947,4701
4,2000,VIRGINIA,51009,AMHERST,4812,6660,11472


In [47]:
md.drop(columns=['Unnamed: 0'], inplace=True)
md.head()

,County Name,County Code,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,0-50K,50K-100K,100K and above,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,Accomack County,1.0,394.0,84.0,32.0,44.0,127.0,1702.0,666.0,532.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
1,Albemarle County,3.0,509.0,328.0,140.0,223.0,43.0,2158.0,1278.0,1329.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
2,Alleghany County,5.0,75.0,113.0,5.0,70.0,127.0,1012.0,447.0,284.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
3,Amelia County,7.0,152.0,212.0,38.0,53.0,28.0,978.0,515.0,388.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
4,Amherst County,9.0,323.0,162.0,85.0,51.0,127.0,1406.0,635.0,461.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496


In [48]:
pop.head()

,YEAR,FIPS,AV0AA,B78AA,AV1AA,AV1AB,AT5AA,AT5AB,AL8AA,AL8AB,...,A88AD,A88AE,AB2AA,BD5AA,AX6AA,CL6AA,AX7AA,AX7AB,BV8AA,BV8AB
0,2000,51001,38305.0,38305.0,18590.0,19715.0,36680.0,1625.0,22535.0,13983.0,...,3781.0,3195.0,34821.0,16309.0,37715.0,6788.0,6788.0,30927.0,957.0,5291.0
1,125,51001,33454.0,33454.0,16230.0,17224.0,31192.0,2262.0,18212.0,12696.0,...,2649.0,4604.0,50250.0,22909.0,33012.0,6678.0,6678.0,26334.0,1002.0,5147.0
2,195,51001,32673.0,32673.0,15934.0,16739.0,30148.0,2525.0,18418.0,11366.0,...,2395.0,5072.0,57711.0,26018.0,32325.0,6141.0,6141.0,26184.0,677.0,6603.0
3,2000,51003,79236.0,79236.0,38002.0,41234.0,73483.0,5753.0,39195.0,33535.0,...,5153.0,13401.0,63407.0,28852.0,77661.0,5232.0,5232.0,72429.0,430.0,8756.0
4,125,51003,99484.0,99484.0,47564.0,51920.0,89589.0,9895.0,46267.0,41638.0,...,3715.0,18225.0,89768.0,37837.0,93607.0,8321.0,8321.0,85286.0,643.0,13285.0


## joining the two dfs on county name

In [49]:
# Step 1: Normalize county names
vdf['county_name'] = vdf['county_name'].str.lower()
md['County Name'] = md['County Name'].str.lower()

# Step 2: Remove "County" from one data frame
md['County Name'] = md['County Name'].str.replace('county', '')

# Make sure each county only has one row
vdf = vdf.drop_duplicates(subset=['county_name'])
md = md.drop_duplicates(subset=['County Name'])

# Step 3: Rename columns
vdf.rename(columns={'county_name': 'county'}, inplace=True)  # Replace old column names with just 'county'
md.rename(columns={'County Name': 'county'}, inplace=True)

# Step 4: Perform the join operation
merged_df = pd.merge(vdf, md, on='county')

# Change county names back to capital letters
merged_df['county'] = merged_df['county'].str.title()

<ipython-input-49-82f6e49a06e0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vdf.rename(columns={'county_name': 'county'}, inplace=True)  # Replace old column names with just 'county'
<ipython-input-49-82f6e49a06e0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  md.rename(columns={'County Name': 'county'}, inplace=True)


In [50]:
merged_df.head()

,year,state,county_fips,county,democrat_votes,republican_votes,total_votes,County Code,White alone,Black or African American alone,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,2020,VIRGINIA,51510,Alexandria City,66240,14544,80784,510.0,1162.0,667.0,...,6.822197,7.257003,7.013915,6.666957,7.125283,6.583409,7.272398,7.104965,6.582025,7.053586
1,2020,VIRGINIA,51520,Bristol City,2313,5347,7660,520.0,127.0,131.0,...,5.521461,6.347389,6.152733,5.337538,5.252273,5.327876,6.476972,6.204558,5.030438,5.225747
2,2020,VIRGINIA,51530,Buena Vista City,825,1863,2688,530.0,73.0,72.0,...,5.533389,5.609472,5.181784,4.406719,4.615121,5.655992,6.274762,5.598422,4.595120,5.501258
3,2020,VIRGINIA,51540,Charlottesville City,20696,3094,23790,540.0,270.0,372.0,...,5.730100,6.513230,6.267201,5.568345,6.251904,5.758902,6.566672,6.156979,5.541264,6.182085
4,2020,VIRGINIA,51550,Chesapeake City,66377,58180,124557,550.0,847.0,540.0,...,6.272877,7.558517,7.427144,6.501290,6.683361,6.317165,7.551712,7.471363,6.712956,6.863803


In [51]:
# Choose only the columns with population, year, and fips code
pop_df = pop[['YEAR', 'FIPS', 'AV0AA']]

# Function that maps years to year ranges
def map_year_to_range(year):
    if 2000 <= year <= 2007:
        return 2000
    elif 2008 <= year <= 2014:
        return 125  # Year range (2008, 2012) represented by 125
    elif 2015 <= year <= 2024:
        return 195  # Year range (2015, 2019) represented by 195

# Apply the function to map years to year ranges
pop_df['YEAR'] = pop_df['YEAR'].apply(map_year_to_range)

# Aggregate population data by county
pop_df = pop_df.groupby('FIPS').agg({'AV0AA': 'sum'}).reset_index()

# Merge the population data with the merged DataFrame (vdf) on 'FIPS' and 'county_fips'
merged_df = pd.merge(merged_df, pop_df, left_on='county_fips', right_on='FIPS', how='left')

# Drop the duplicate 'FIPS' column if necessary
merged_df.drop(columns=['FIPS'], inplace=True)

# Display the resulting DataFrame
merged_df

<ipython-input-51-54108c213a58>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_df['YEAR'] = pop_df['YEAR'].apply(map_year_to_range)


,year,state,county_fips,county,democrat_votes,republican_votes,total_votes,County Code,White alone,Black or African American alone,...,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log,AV0AA
0,2020,VIRGINIA,51510,Alexandria City,66240,14544,80784,510.0,1162.0,667.0,...,7.257003,7.013915,6.666957,7.125283,6.583409,7.272398,7.104965,6.582025,7.053586,426233.0
1,2020,VIRGINIA,51520,Bristol City,2313,5347,7660,520.0,127.0,131.0,...,6.347389,6.152733,5.337538,5.252273,5.327876,6.476972,6.204558,5.030438,5.225747,52026.0
2,2020,VIRGINIA,51530,Buena Vista City,825,1863,2688,530.0,73.0,72.0,...,5.609472,5.181784,4.406719,4.615121,5.655992,6.274762,5.598422,4.595120,5.501258,19512.0
3,2020,VIRGINIA,51540,Charlottesville City,20696,3094,23790,540.0,270.0,372.0,...,6.513230,6.267201,5.568345,6.251904,5.758902,6.566672,6.156979,5.541264,6.182085,135368.0
4,2020,VIRGINIA,51550,Chesapeake City,66377,58180,124557,550.0,847.0,540.0,...,7.558517,7.427144,6.501290,6.683361,6.317165,7.551712,7.471363,6.712956,6.863803,662399.0
5,2020,VIRGINIA,51570,Colonial Heights City,2972,6007,8979,570.0,155.0,183.0,...,6.165418,5.888878,5.117994,5.231109,5.017280,6.282267,6.098074,4.852030,5.198497,51751.0
6,2020,VIRGINIA,51580,Covington City,964,1580,2544,580.0,61.0,108.0,...,5.676754,5.433722,4.143135,5.680173,5.932245,5.820083,5.552960,4.219508,5.347108,17804.0
7,2020,VIRGINIA,51590,Danville City,11710,7428,19138,590.0,273.0,79.0,...,6.851185,6.444131,5.451038,5.652489,5.991465,6.920672,6.712956,5.347108,5.537334,132666.0
8,2020,VIRGINIA,51595,Emporia City,1612,754,2366,595.0,319.0,302.0,...,6.120297,4.941642,4.234107,4.859812,5.247024,5.897154,5.433722,4.262680,4.499810,16932.0
9,2020,VIRGINIA,51600,Fairfax City,9174,4007,13181,600.0,359.0,122.0,...,5.905362,6.137727,5.497168,6.131226,5.170484,6.098074,6.086775,5.666427,5.902633,67741.0


In [52]:
merged_df.to_csv("real_combined_data.csv")


In [53]:
merged_df.shape

(38, 61)

In [54]:
vdf.shape

(167, 7)

In [55]:
md.shape

(134, 54)